# Conclusions


## Contributors

Additional annotated and unannotated work can be find in our individual folders, but not all of this made it into the final report:

[Alex Caian](https://github.com/Galeforse/DST-Assessment-04/tree/main/Alex%20Caian)
    
[Matt Corrie](https://github.com/Galeforse/DST-Assessment-04/tree/main/Matt%20Corrie)

[Gabriel Grant](https://github.com/Galeforse/DST-Assessment-04/tree/main/Gabriel%20Grant)